In [16]:
import kagglehub
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import sklearn as sk
from sklearn.model_selection import train_test_split
import splitfolders
import matplotlib.pyplot as plt

# Download chest x-ray dataset

In [3]:
# # Download latest version
# path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
# work_dir = "/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/DATA"

# print(work_dir, path)

# Data Split



In [ ]:
# adnorm_im_datapath = "/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/all_data"
# splitfolders.ratio(adnorm_im_datapath, output="/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/all_data/splits", seed=1337, ratio=(.8, 0.1,0.1))

Copying files: 5856 files [00:35, 164.09 files/s]


In [4]:
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
# from keras.preprocessing.image import ImageDataGenerator
# from keras.optimizers import SGD

# def Build_CNN_Model():
#     model = Sequential()
#     # Add a 2D Convolutional layer with 32 filters and a 3x3 kernel
#     model.add(Conv2D(32, (3, 3), padding='same', input_shape=(224, 224, 1)))
#     # Remove all values that are negative and change to zero
#     model.add(Activation('relu'))
#     # Add a 2D MaxPooling layer with a 2x2 pool size(reduced function and take the maximum values in the pool)
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     # -----------------------------------------------------
#     # Add a 2D Convolutional layer with 64 filters and a 3x3 kernel
#     model.add(Conv2D(64, (3, 3), padding='same'))
#     # Remove all values that are negative and change to zero
#     model.add(Activation('relu'))
#     # Add a 2D MaxPooling layer with a 2x2 pool size(reduced function and take the maximum values in the pool)
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     # Add a 2D Convolutional layer with 128 filters and a 3x3 kernel

#     # -----------------------------------------------------
#     model.add(Conv2D(128, (3, 3), padding='same'))
#     # Remove all values that are negative and change to zero
#     model.add(Activation('relu'))
#     # Add a 2D MaxPooling layer with a 2x2 pool size(reduced function and take the maximum values in the pool)
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     # -----------------------------------------------------
#     # Apply flattening to the model - reduce from high deminsionallty to 1D(flat column)
#     model.add(Flatten())
#     # Add a dense layer with 1000 neurons
#     model.add(Dense(1000))
#     # Add a relu activation function to the dense layer
#     model.add(Activation('relu'))
#     # Add softmax activation function to the output layer for classification
#     model.add(Dense(2))
#     model.add(Activation('softmax'))
#     # Display the model summary
#     model.summary()

#     # Compile the model with categorical crossentropy loss function and SGD optimizer
#     opt = SGD(learning_rate=0.001)
#     model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
#     return model

# Build_CNN_Model()

# Basic CNN model to classify abnormal chest x-rays

In [17]:
def Build_CNN_Model():
    model = Sequential()
    # Add a 2D Convolutional layer with 32 filters and a 3x3 kernel
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(224, 224, 1)))
    # Remove all values that are negative and change to zero
    model.add(Activation('relu'))
    # Add a 2D MaxPooling layer with a 2x2 pool size (reduced function and take the maximum values in the pool)
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # -----------------------------------------------------
    # Add a 2D Convolutional layer with 64 filters and a 3x3 kernel
    model.add(Conv2D(64, (3, 3), padding='same'))
    # Remove all values that are negative and change to zero
    model.add(Activation('relu'))
    # Add a 2D MaxPooling layer with a 2x2 pool size (reduced function and take the maximum values in the pool)
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # -----------------------------------------------------
    # Add a 2D Convolutional layer with 128 filters and a 3x3 kernel
    model.add(Conv2D(128, (3, 3), padding='same'))
    # Remove all values that are negative and change to zero
    model.add(Activation('relu'))
    # Add a 2D MaxPooling layer with a 2x2 pool size (reduced function and take the maximum values in the pool)
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Apply flattening to the model - reduce from high deminsionallty to 1D(flat column)
    model.add(Flatten())
    # Add a dense layer with 1000 neurons
    model.add(Dense(1000))
    # Add a relu activation function to the dense layer
    model.add(Activation('relu'))
    # Add softmax activation function to the output layer for classification
    model.add(Dense(2))
    model.add(Activation('softmax'))

    # Display the model summary
    model.summary()

    # Compile the model with categorical crossentropy loss function and SGD optimizer
    opt = SGD(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Example of using GPU
if len(tf.config.experimental.list_physical_devices('GPU')) > 0:
    with tf.device('/GPU:0'):
        model = Build_CNN_Model()
else:
    print("No GPU available, building model on CPU")
    model = Build_CNN_Model()

Num GPUs Available:  1
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 224, 224, 32)      320       
                                                                 
 activation_10 (Activation)  (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 activation_11 (Activation)  (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                           

In [19]:
def Train_CNN_Model(model):
    # Model Parameters
    batch_size = 32
    epochs = 50
    # Data Augmentation fro grayscale xray images
    # Data normalize by scaleing and shifting and normalizing the data
    # - Scaling: making the pixel repersented as a float between 0 and 255 to a float between 0 and 1
    # - Zero Mean Shifting: shifting the data to have a mean of 0
    # - Normalization: scaling the data to have a standard deviation of 1 using the unit variance
    # from keras.preprocessing.image import ImageDataGenerator
    train_datagen =  ImageDataGenerator(rescale=1./255,featurewise_center=True, featurewise_std_normalization=True)
    valid_datagen =  ImageDataGenerator(rescale=1./255,featurewise_center=True, featurewise_std_normalization=True)

    # Load the training data
    train_generator = train_datagen.flow_from_directory(
            '/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/all_data/splits/train',
            classes=['normal', 'adnormal'],
            target_size=(224, 224), 
            color_mode='grayscale',
            batch_size=batch_size,
            class_mode='categorical')

    # Load the validation data
    valid_generator = valid_datagen.flow_from_directory(
            '/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/all_data/splits/val',
            classes=['normal', 'adnormal'],
            target_size=(224, 224),
            color_mode='grayscale',
            batch_size=batch_size,
            class_mode='categorical')
    
    
    history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=epochs, validation_data=valid_generator, validation_steps=len(valid_generator), verbose=1)

    # Accuracy and Loss
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')  
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Save the model
    model.save('chest_xray_cnn.h5')

# Train Classifiyer

In [14]:
model = Build_CNN_Model()
Train_CNN_Model(model)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 32)      320       
                                                                 
 activation_5 (Activation)   (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 activation_6 (Activation)   (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                  

/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/tf-gpu-env/lib/python3.10/site-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/Users/mmckay/Desktop/Skill_Development/chest_xray_CNN_training/tf-gpu-env/lib/python3.10/site-packages/keras/preprocessing/image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/50


2024-12-05 21:15:11.987985: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-12-05 21:15:12.637460: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-12-05 21:15:13.286385: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignVariableOp.


40/40 [==============================] - ETA: 0s - loss: 0.2995 - accuracy: 1.0000

2024-12-05 21:15:36.690585: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-12-05 21:15:36.829015: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


40/40 [==============================] - 85s 2s/step - loss: 0.2995 - accuracy: 1.0000 - val_loss: 0.0960 - val_accuracy: 1.0000
Epoch 2/50
40/40 [==============================] - 31s 776ms/step - loss: 0.0538 - accuracy: 1.0000 - val_loss: 0.0287 - val_accuracy: 1.0000
Epoch 3/50
40/40 [==============================] - 28s 691ms/step - loss: 0.0206 - accuracy: 1.0000 - val_loss: 0.0143 - val_accuracy: 1.0000
Epoch 4/50
40/40 [==============================] - 28s 692ms/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 0.0089 - val_accuracy: 1.0000
Epoch 5/50
40/40 [==============================] - 29s 708ms/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 0.0063 - val_accuracy: 1.0000
Epoch 6/50
40/40 [==============================] - 28s 702ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 7/50
40/40 [==============================] - 33s 832ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 8/50
40

ModuleNotFoundError: No module named 'matplotlib'

In [20]:
#! Add data from training to validation set to increase the size of the validation set - because overfitting is a major issue with this dataset

In [ ]:
# Organize all the data into a single directory
# - Create a new directory called data
# - train test validation split for normal and abnormal directories